In [9]:
import pymysql.cursors

db = pymysql.connect(host='13.230.161.175',
                             user='public-root',
                             password='thisisasamplepassword',                             
                             db='game_chatbot',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

def search_db(keyword):
    try:
        cursor = db.cursor()#建立資料庫游標
        sql = "select game_name \
                from game\
                where game_ID in (select game_ID\
                    from game_has_type,type\
                    where type_name = '"+keyword+"' )" #下指令
        cursor.execute(sql)
        rows = cursor.fetchall()
        return str(rows[0])
    except Exception as e:
        print("Exeception occured:{}".format(e))
    finally:
        db.close()

In [ ]:
import json
from flask import Flask, request, make_response, jsonify
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pymysql.cursors

app = Flask(__name__)


@app.route("/")
def verify():
    return "Hello world", 200


@app.route('/', methods=['POST'])#Dialogflow端
def webhook():
    req = request.get_json(silent=True, force=True)
    print(req)
    if req['queryResult']['parameters']['game_type'] != None:
        keyword = req['queryResult']['parameters']['game_type']
    res = {"fulfillmentText": search_db(keyword)}
    res = json.dumps(res,indent = 4)
    print(res)
    return res


if __name__ == '__main__':
    db = pymysql.connect(host='13.230.161.175',#連結MySQL
                             user='public-root',
                             password='thisisasamplepassword',                             
                             db='game_chatbot',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    app.run(port=5000)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Dec/2018 17:02:59] "POST / HTTP/1.1" 200 -


{'responseId': '6c2a7c8f-0dec-4548-a210-0d516f26d07f', 'queryResult': {'queryText': '恐怖遊戲', 'action': 'action', 'parameters': {'game_type': '驚悚'}, 'allRequiredParamsPresent': True, 'fulfillmentMessages': [{'text': {'text': ['']}}], 'intent': {'name': 'projects/deepbowl-e7505/agent/intents/71659b1b-175b-465a-b508-067b18a7e475', 'displayName': 'GameChatBot'}, 'intentDetectionConfidence': 0.91, 'languageCode': 'zh-cn'}, 'originalDetectIntentRequest': {'payload': {}}, 'session': 'projects/deepbowl-e7505/agent/sessions/b7045468-a131-3321-8af9-585e4a41a25f'}
{
    "fulfillmentText": "{'game_name': 'STEINS;GATE'}"
}
